In [1]:
# import packages

import lyricsgenius
import pandas as pd
import requests

from textblob import TextBlob 
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

import spacy
from spacy.lang.en import English
nlp = English()
nlp.max_length = 10000000

from gensim.summarization import keywords

[nltk_data] Downloading package stopwords to /Users/0x23/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# search_term = str(input('Enter artist name'))
search_term = 'Dua Lipa'

In [3]:
client_access_token = 'H1FwqolIMEafwAsR6iQ_zBZNrteSUUX5BBGAjNo5oq57TMwW13cg70vlpVj1QJJc'

In [4]:
genius_search_url = f"http://api.genius.com/search?q={search_term}&access_token={client_access_token}"
response = requests.get(genius_search_url)
json_data = response.json()

In [14]:
songs = []
for song in json_data['response']['hits']:
    songs.append([song['result']['title'], song['result']['primary_artist']['name']])
    
#Make a Pandas dataframe from a list
song_df = pd.DataFrame(songs)
song_df.columns = ['song','artist']
song_df

,song,artist
0,New Rules,Dua Lipa
1,Scared to Be Lonely,Martin Garrix & Dua Lipa
2,Don’t Start Now,Dua Lipa
3,IDGAF,Dua Lipa
4,One Kiss,Calvin Harris & Dua Lipa
5,Blow Your Mind (Mwah),Dua Lipa
6,Levitating,Dua Lipa
7,Break My Heart,Dua Lipa
8,Kiss and Make Up,Dua Lipa & BLACKPINK
9,Be the One,Dua Lipa


In [6]:
# Function to return lyrics of each song using Genius API
def get_lyrics(title, artist):
    try:
        return genius.search_song(title, artist).lyrics
    except:
        return 'not found'

# Function to return sentiment score of each song
def get_lyric_sentiment(lyrics):
	analysis = TextBlob(lyrics) 
	return analysis.sentiment.polarity


# Function to preprocess text
def preprocess(text):
    # Create Doc object
    doc = nlp(text, disable=['ner', 'parser'])
    # Generate lemmas
    lemmas = [token.lemma_ for token in doc]
    # Remove stopwords and non-alphabetic characters
    a_lemmas = [lemma for lemma in lemmas 
            if lemma.isalpha() and lemma not in stopwords]
    return ' '.join(a_lemmas)


# Extract Keywords from text
def return_keywords(texts):
    xkeywords = []
    values = keywords(text=preprocess(texts),split='\n',scores=True)
    for x in values[:10]:
        xkeywords.append(x[0])
    try:
        return xkeywords 
    except:
        return "no content"

In [8]:
song_df

,song,artist,Lyrics


In [9]:
genius = lyricsgenius.Genius(client_access_token)

In [11]:
# Use get_lyrics funcion to get lyrics for every song in dataset
lyrics = song_df.apply(lambda row: get_lyrics(row['song'], row['artist']), axis =1)
song_df['Lyrics'] = lyrics
song_df = song_df.drop(song_df[song_df['Lyrics'] == 'not found'].index) #

In [13]:
# Use get_lyric_sentiment to get sentiment score for all the song lyrics
sentiment = song_df.apply(lambda row: get_lyric_sentiment(row['Lyrics']), axis =1)
song_df['Sentiment'] = sentiment

ValueError: Wrong number of items passed 3, placement implies 1

In [ ]:
song_df

,song,artist,Lyrics,Sentiment
0,New Rules,Dua Lipa,"[Intro]\nOne, one, one, one, one\n\n[Verse 1]\...",0.129119
1,Scared to Be Lonely,Martin Garrix & Dua Lipa,[Verse 1]\nIt was great at the very start\nHan...,0.057258
2,Don’t Start Now,Dua Lipa,[Intro]\nIf you don't wanna see me\n\n[Verse 1...,0.047727
3,IDGAF,Dua Lipa,[Verse 1]\nYou call me all friendly\nTellin' m...,-0.042188
4,One Kiss,Calvin Harris & Dua Lipa,[Intro]\nOne kiss is all it takes\nFallin' in ...,0.466667
5,Blow Your Mind (Mwah),Dua Lipa,[Verse 1]\nI know it's hot\nI know we've got\n...,-0.023333
6,Levitating,Dua Lipa,"[Verse 1]\nIf you wanna run away with me, I kn...",0.639423
7,Break My Heart,Dua Lipa,[Verse 1]\nI've always been the one to say the...,0.271739
8,Kiss and Make Up,Dua Lipa & BLACKPINK,[Verse 1: Dua Lipa]\nWe haven't talked all mor...,-0.173913
9,Be the One,Dua Lipa,"[Verse 1]\nI see the moon, I see the moon, I s...",-0.218519


----------